<a href="https://colab.research.google.com/github/AjayKumarGogineni777/Dynamic-Convolution-LSTM/blob/master/DCNN_Search_Snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision.gan import *

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
file = "C:/Users/Raj kishor/Desktop/1D/Short text clusterning/data-web-snippets/train.txt"
corpus = open(file,encoding='utf8')
corpus = corpus.readlines()
#docs = corpus.split("\n") 

In [0]:
corpus[0]

'manufacture manufacturer directory directory china taiwan products manufacturers directory- taiwan china products manufacturer direcory exporter directory supplier directory suppliers business\n'

In [0]:
!pip install nltk

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Raj
[nltk_data]     kishor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
import nltk
from fastai.text import *
from nltk import tokenize
a1 = tokenize.sent_tokenize(corpus[0])

In [0]:
len(a1[0].split(' ')[-1])

8

In [0]:
a2 = tokenize.sent_tokenize(corpus[10])

In [0]:
a1[0][0:-9]

'manufacture manufacturer directory directory china taiwan products manufacturers directory- taiwan china products manufacturer direcory exporter directory supplier directory suppliers'

In [0]:
last = '\n'
a1[0][0:-9]+last

'manufacture manufacturer directory directory china taiwan products manufacturers directory- taiwan china products manufacturer direcory exporter directory supplier directory suppliers\n'

In [0]:
len(corpus)

10060

In [0]:
labels = []
text = []
last = '\n'
for i in range(len(corpus)):
  a1 = tokenize.sent_tokenize(corpus[i])
  #print(i)
  temp_label = a1[0].split(' ')[-1]
  
  length = len(a1[0].split(' ')[-1])
  temp_text = a1[0][0:-1*(length+1)]+last
  labels.append(temp_label)
  text.append(temp_text)
  

In [0]:
labels

In [0]:
len(text)

10060

In [0]:
import pandas as pd
df = pd.DataFrame(list(zip(labels, text)), 
               columns =['labels', 'text']) 
df[0:5]

,labels,text
0,business,manufacture manufacturer directory directory c...
1,business,empmag electronics manufacturing procurement h...
2,business,dfma truecost paper true cost overseas manufac...
3,business,thomasnet thomasnet cnc machining metal stampi...
4,business,crnano products nanotechnology products molecu...


In [0]:
df[0:1]['text']

0    manufacture manufacturer directory directory c...
Name: text, dtype: object

In [0]:
from sklearn.feature_extraction.text import CountVectorizer 
veczr = CountVectorizer(binary = 'True')

In [0]:
corpus[0:2]

['manufacture manufacturer directory directory china taiwan products manufacturers directory- taiwan china products manufacturer direcory exporter directory supplier directory suppliers business\n',
 'empmag electronics manufacturing procurement homepage electronics manufacturing procurement magazine procrement power products production essentials data management business\n']

In [0]:
trn_term_doc = veczr.fit_transform(corpus)

In [0]:
trn_term_doc.shape


(10060, 25219)

In [0]:
from scipy.sparse.csc import csc_matrix
A = csc_matrix(trn_term_doc[0])

In [0]:
A.tocsr()

<1x25219 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [0]:
kav = A[0,:].A
kav[0]

array([0, 0, 0, 0, ..., 0, 0, 0, 0], dtype=int64)

In [0]:
import numpy as np### Similarity Matrix

X = np.zeros((10060, 25219))
for i in range(10060):
    ma = trn_term_doc[i].tocsr()
    te = ma[0,:].A
    te = te[0]
    X[i,:] = te

In [0]:
X

array([[ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.]])

In [0]:
X[0]

array([ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.])

In [0]:
def distance(a,b):
    count = 0
    for i in range(10060):
        if a[i] == 1 and b[i] == 1:
            count = count+1 

In [0]:
import spacy
nlp = spacy.load('en')
doc1 = nlp(corpus[0])
doc2 = nlp(corpus[1])
doc1.similarity(doc2)

In [0]:
corpus[0]

'manufacture manufacturer directory directory china taiwan products manufacturers directory- taiwan china products manufacturer direcory exporter directory supplier directory suppliers business\n'

In [0]:
corpus[3]

'thomasnet thomasnet cnc machining metal stamping gaskets fasteners searchable database information products services cad drawings business\n'

In [0]:
x = X[0]
y = X[3]
p = x[y ==1].sum(0)
p

2.0

In [0]:
### Finding k = 25 neighbours
sim = np.zeros((10060,10060))
for i in range(10060):
    initial = X[i]
    for j in range(i+1, 10060):
        count = 0
        nex = X[j]
        #count = 
        sim[i][j] = initial[nex == 1].sum(0)

In [0]:
sim2 = sim
for i1 in range(10060):
    for j1 in range(10060):
        if i1>j1:
            sim2[i1][j1] = sim[j1][i1]

In [0]:
sim2

array([[  0.,   2.,   4.,   2., ...,   0.,   0.,   0.,   0.],
       [  2.,   0.,   3.,   2., ...,   0.,   0.,   0.,   0.],
       [  4.,   3.,   0.,   2., ...,   0.,   0.,   0.,   0.],
       [  2.,   2.,   2.,   0., ...,   0.,   0.,   0.,   0.],
       ..., 
       [  0.,   0.,   0.,   0., ...,   0.,   5.,   4.,   2.],
       [  0.,   0.,   0.,   0., ...,   5.,   0.,  11.,   2.],
       [  0.,   0.,   0.,   0., ...,   4.,  11.,   0.,   3.],
       [  0.,   0.,   0.,   0., ...,   2.,   2.,   3.,   0.]])

In [0]:
sorted1 = np.sort(sim2[0])
sorted1

array([ 0.,  0.,  0.,  0., ...,  4.,  4.,  5.,  6.])

In [0]:
sorted1[-3]

4.0

In [0]:
kw = np.where(sim2[0] == sorted1[-1])
for elem in kw[0]:
    print(elem)

13


In [0]:
kw = list(kw)
kwa = kw[0:3]

In [0]:
kw = np.where(sim2[1] == np.sort(sim2[-1]))
kw2 = np.where(sim2[1] ==np.sort(sim2[-2]))

In [0]:
kw[0].shape

(6558,)

In [0]:
tes = [1,3,2,0]
np.argsort(tes)

array([3, 0, 2, 1], dtype=int64)

In [0]:
np.argsort(tes)[-4:]

array([3, 0, 2, 1], dtype=int64)

In [0]:
np.append(kw,kw2)

array([   1, 1200, 1201, 1202, ..., 9200, 9256, 9291, 9300], dtype=int64)

In [0]:
kw = np.where(sim2[0] == sorted1[-4])
kw

(array([378], dtype=int64),)

In [0]:
np.argsort(sim2[0])

array([   0, 6588, 6589, 6590, ...,    2,   16,  378,   13], dtype=int64)

In [0]:
sim2[5]

array([ 3.,  2.,  3.,  2., ...,  0.,  0.,  0.,  0.])

In [0]:
sigma_sq = 1
np.exp(-1*(abs(sim2[0]-sim2[1])*abs(sim2[0]-sim2[1]))/2*sigma_sq).sum()

9602.3594700692738

In [0]:
#### final exp neighbouring matrix
neigh = np.zeros((10060,10060))
sigma_sq = 1
for i in range(10060):
    initial_n = sim2[i]
    sorted1 = np.sort(sim2[i])
    last = -1
    neighbours = 0
    num=0
    
    lis = np.argsort(sim2[i])
    tak = lis[-25:]
    for elem in tak:
        term = np.exp(-1*(abs(sim2[i]-sim2[elem])*abs(sim2[i]-sim2[elem]))/2*sigma_sq).sum()
        neigh[i][elem] = term
    

In [0]:
np.count_nonzero(neigh[10000])

25

In [0]:
W = neigh
D = np.zeros((10060,10060))
for i in range(10060):
    D[i][i] = np.sum(W[i,:])

In [0]:
L = D-W

In [0]:
from sklearn.manifold import SpectralEmbedding

In [0]:
#### Embedding of size 64
embedding = SpectralEmbedding(n_components = 64)
transformed_64 = embedding.fit_transform(L)

In [0]:
transformed.shape

(10060, 10)

In [0]:
np.savetxt('C:/Users/Raj kishor/Desktop/1D/Short text clusterning/data-web-snippets/transformed_64.txt',transformed_64)
np.save('C:/Users/Raj kishor/Desktop/1D/Short text clusterning/data-web-snippets/transformed_64',transformed_64)

In [0]:
np.count_nonzero(D[1000])

1

In [0]:
ta =[1,2,3],[3,4,5],[8,9,10]

In [0]:
for i range(3):
    
    

([1, 2, 3], [3, 4, 5], [8, 9, 10])

In [0]:
np.where(y==1)

(array([ 2980,  3058,  4120,  5480,  6537,  7980,  8982, 11065, 13237, 13988, 17704, 19835, 20059, 21208, 22522], dtype=int64),)

In [0]:
### Finding k = 25 neighbours
sim = np.zeros((10060,10060))
for i in range(10060):
    initial = X[i]
    for j in range(i+1, 10060):
        count = 0
        nex = X[j]
        for k in range(1):
            if initial[k] == 1 and nex[k] == 1:
                count = count+1
    sim[i][j] = count            

In [0]:
sim[0]

array([ 0.,  0.,  0.,  0., ...,  0.,  0.,  0.,  0.])

In [0]:
import numpy as np
np.array(trn_term_doc[0])

array(<1x25219 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>, dtype=object)

In [0]:
print(trn_term_doc[0])

  (0, 2980)	1
  (0, 21723)	1
  (0, 21722)	1
  (0, 7771)	1
  (0, 6130)	1
  (0, 13443)	1
  (0, 17704)	1
  (0, 21967)	1
  (0, 3777)	1
  (0, 6145)	1
  (0, 13442)	1
  (0, 13440)	1


In [0]:
df[:]['text'][1]

'empmag electronics manufacturing procurement homepage electronics manufacturing procurement magazine procrement power products production essentials data management\n'

In [0]:
data = [line.strip() for line in open(file, 'r')]
texts = [[word.lower() for word in text.split()] for text in data]

In [0]:
data[0]

'manufacture manufacturer directory directory china taiwan products manufacturers directory- taiwan china products manufacturer direcory exporter directory supplier directory suppliers business'

In [0]:
trn_term_doc = veczr.fit_transform(data[0:10])

In [0]:
t1 = corpus[0:10]
trn_term_doc = veczr.fit_transform(t1)

In [0]:
[corpus[0:10]]

[['manufacture manufacturer directory directory china taiwan products manufacturers directory- taiwan china products manufacturer direcory exporter directory supplier directory suppliers business\n',
  'empmag electronics manufacturing procurement homepage electronics manufacturing procurement magazine procrement power products production essentials data management business\n',
  'dfma truecost paper true cost overseas manufacture product design costs manufacturing products china manufacturing redesigned product china save business\n',
  'thomasnet thomasnet cnc machining metal stamping gaskets fasteners searchable database information products services cad drawings business\n',
  'crnano products nanotechnology products molecular manufacturing product structural design simplest products software-controlled extra cost manufacturing business\n',
  'epa gov wastewise wrr buyman buy manufacture recycled wastewise epa organizations manufactured products recycled content percentage recycled

In [0]:
a1[0].split(' ')[0:-1]

['americangymnasticsclub',
 'american',
 'gymnastic',
 'club',
 'recreational',
 'gymnastics',
 'boys',
 'girls',
 'schedule',
 'fees',
 'programs',
 'calendar',
 'birthday',
 'parties',
 'camps',
 'staff']

In [0]:
a2 = tokenize.sent_tokenize(corpus[0])
np.unique(a2[0].split(' ')[0:-1])

array(['china', 'direcory', 'directory', 'directory-', 'exporter', 'manufacture', 'manufacturer', 'manufacturers',
       'products', 'supplier', 'suppliers', 'taiwan'], dtype='<U13')

In [0]:
k2 = np.unique(a2[0].split(' ')[0:-1])
len(k2)

12

In [0]:
corpus[0]

'manufacture manufacturer directory directory china taiwan products manufacturers directory- taiwan china products manufacturer direcory exporter directory supplier directory suppliers business\n'

In [0]:
#unique = []
unique = np.unique(a2[0].split(' ')[0:-1])
unique = np.ndarray.tolist(unique)
for i in range(len(corpus)):
  a1 = tokenize.sent_tokenize(corpus[i])
  #print(i)
  temp_label = a1[0].split(' ')[-1]
  
  length = len(a1[0].split(' ')[-1])
  temp_text = a1[0][0:-1*(length+1)]
  k1 = np.unique(a1[0].split(' ')[0:-1])
  print(k1)
  print(unique)
  
  k1 = np.array(k1)
  for j in range(len(k1)):
    if (k1[j] not in unique):
      print(k1[j])
      
      unique.append(k1[j])


['china' 'direcory' 'directory' 'directory-' 'exporter' 'manufacture' 'manufacturer' 'manufacturers' 'products'
 'supplier' 'suppliers' 'taiwan']
['china', 'direcory', 'directory', 'directory-', 'exporter', 'manufacture', 'manufacturer', 'manufacturers', 'products', 'supplier', 'suppliers', 'taiwan']
['data' 'electronics' 'empmag' 'essentials' 'homepage' 'magazine' 'management' 'manufacturing' 'power' 'procrement'
 'procurement' 'production' 'products']
['china', 'direcory', 'directory', 'directory-', 'exporter', 'manufacture', 'manufacturer', 'manufacturers', 'products', 'supplier', 'suppliers', 'taiwan']
data
electronics
empmag
essentials
homepage
magazine
management
manufacturing
power
procrement
procurement
production
['china' 'cost' 'costs' 'design' 'dfma' 'manufacture' 'manufacturing' 'overseas' 'paper' 'product' 'products'
 'redesigned' 'save' 'true' 'truecost']
['china', 'direcory', 'directory', 'directory-', 'exporter', 'manufacture', 'manufacturer', 'manufacturers', 'products

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
unique

['china',
 'direcory',
 'directory',
 'directory-',
 'exporter',
 'manufacture',
 'manufacturer',
 'manufacturers',
 'products',
 'supplier',
 'suppliers',
 'taiwan',
 'data',
 'electronics',
 'empmag',
 'essentials',
 'homepage',
 'magazine',
 'management',
 'manufacturing',
 'power',
 'procrement',
 'procurement',
 'production',
 'cost',
 'costs',
 'design',
 'dfma',
 'overseas',
 'paper',
 'product',
 'redesigned',
 'save',
 'true',
 'truecost',
 'cad',
 'cnc',
 'database',
 'drawings',
 'fasteners',
 'gaskets',
 'information',
 'machining',
 'metal',
 'searchable',
 'services',
 'stamping',
 'thomasnet',
 'crnano',
 'extra',
 'molecular',
 'nanotechnology',
 'simplest',
 'software-controlled',
 'structural',
 'buy',
 'buyman',
 'content',
 'epa',
 'gov',
 'manufactured',
 'organizations',
 'percentage',
 'recycled',
 'wastewise',
 'wrr',
 'cement',
 'concreteproducts',
 'dcsp',
 'health',
 'index',
 'issues',
 'osha',
 'phases',
 'portland',
 'pre-cast',
 'safety',
 'topics',
 'cal

In [0]:
len(unique)

26265

In [0]:
corpus[0][0]

'm'

In [0]:
### Term Document Matrix
mat = np.zeros((len(corpus),len(unique)))
for i in range(len(corpus)):
  for j in range(len(unique)):
    a1 = tokenize.sent_tokenize(corpus[i])
    #print(i)
    temp_label = a1[0].split(' ')[-1]
  
    length = len(a1[0].split(' ')[-1])
    temp_text = a1[0][0:-1*(length+1)].split(' ')
    length2 = len(temp_text)
    for k in range(length2):
      if temp_text[k] == unique[j]:
        mat[i][j] = 1
    

In [0]:
np.save('content/gdrive/My Drive/DCNN/Term_doc_matrix/geekfile', mat) 

In [0]:
temp_text = a1[0][0:-1*(length+1)].split(' ')
temp_text

['americangymnasticsclub',
 'american',
 'gymnastic',
 'club',
 'recreational',
 'gymnastics',
 'boys',
 'girls',
 'schedule',
 'fees',
 'programs',
 'calendar',
 'birthday',
 'parties',
 'camps',
 'staff']

In [0]:
b1 = np.ndarray.tolist(unique)
b1.append(k1[0])

In [0]:
b1

['china',
 'direcory',
 'directory',
 'directory-',
 'exporter',
 'manufacture',
 'manufacturer',
 'manufacturers',
 'products',
 'supplier',
 'suppliers',
 'taiwan',
 'data']

In [0]:
aa = china not in k2
#print(1)

In [0]:
aa

False

In [0]:
if (china not in k2):
  print(2)
else:
  print(3)

3


In [0]:
k2[0]

'china'

In [0]:
#### term document matrix 
#!pip install textmining
pip install textmining==1.0

In [0]:
!pip install p01.stemmer
import p01.stemmer as stemmer

In [0]:
from nltk.stem import *
from nltk.stem import stemmer

In [0]:
import textmining
import glob

tdm = textmining.TermDocumentMatrix()
file = "/content/gdrive/My Drive/DCNN/Term_doc_matrix/train.txt"
content = open(f).read()
content = content.replace('\n', ' ')
tdm.add_doc(content)


In [0]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 

def fn_tdm_df(docs, xColNames = None, **kwargs):
    ''' create a term document matrix as pandas DataFrame
    with **kwargs you can pass arguments of CountVectorizer
    if xColNames is given the dataframe gets columns Names'''

    #initialize the  vectorizer
    vectorizer = CountVectorizer(**kwargs)
    x1 = vectorizer.fit_transform(docs)
    #create dataFrame
    df = pd.DataFrame(x1.toarray().transpose(), index = vectorizer.get_feature_names())
    if xColNames is not None:
        df.columns = xColNames

    return df
DIR = 'C:/Data/'

def fn_CorpusFromDIR(xDIR):
    ''' functions to create corpus from a Directories
    Input: Directory
    Output: A dictionary with 
             Names of files ['ColNames']
             the text in corpus ['docs']'''
    import os
    Res = dict(docs = [open(os.path.join(xDIR,f)).read() for f in os.listdir(xDIR)],
               ColNames = map(lambda x: 'P_' + x[0:6], os.listdir(xDIR)))
    return Res  